In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats


In [2]:
df = pd.read_csv('cleaned_marketing_data.csv')

In [3]:
print(df.describe())

                 ID   Year_Birth         Income      Kidhome     Teenhome  \
count   2240.000000  2240.000000    2240.000000  2240.000000  2240.000000   
mean    5592.159821  1968.805804   51876.364334     0.444196     0.506250   
std     3246.662198    11.984069   20938.537566     0.538398     0.544538   
min        0.000000  1893.000000    1730.000000     0.000000     0.000000   
25%     2828.250000  1959.000000   35538.750000     0.000000     0.000000   
50%     5458.500000  1970.000000   51371.635237     0.000000     0.000000   
75%     8427.750000  1977.000000   68289.750000     1.000000     1.000000   
max    11191.000000  1996.000000  117416.250000     2.000000     2.000000   

           Recency     MntWines    MntFruits  MntMeatProducts  \
count  2240.000000  2240.000000  2240.000000      2240.000000   
mean     49.109375   303.935714    26.302232       166.950000   
std      28.962453   336.597393    39.773434       225.715373   
min       0.000000     0.000000     0.000000  

In [33]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
X = df.drop(['Response', 'ID', 'Dt_Customer'], axis=1)  # drop target and non-predictive IDs/dates
y = df['Response']

# Auto-detect column types
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = X.select_dtypes(include=['object', 'category']).columns.tolist()

In [34]:
numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

In [42]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))
])

# Train model
pipeline.fit(X_train, y_train)

,steps,"[('preprocessor', ...), ('classifier', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [39]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import cross_val_score

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

models = {
    'LogisticRegression': LogisticRegression(max_iter=1000),
    'RandomForest': RandomForestClassifier(n_estimators=100, random_state=42),
    'GradientBoosting': GradientBoostingClassifier(random_state=42)
}

for name, model in models.items():
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', model)
    ])
    scores = cross_val_score(pipeline, X, y, cv=5, scoring='accuracy')
    print(f"{name} Accuracy: {scores.mean():.4f} (+/- {scores.std():.4f})")


LogisticRegression Accuracy: 0.8714 (+/- 0.0397)
RandomForest Accuracy: 0.7179 (+/- 0.2665)
GradientBoosting Accuracy: 0.5835 (+/- 0.2398)


In [43]:
import joblib

# Save entire pipeline (preprocessor + model)
joblib.dump(pipeline, 'marketing_model.pkl')

['marketing_model.pkl']

In [46]:
# Load your trained model
model = joblib.load('marketing_model.pkl')

# Assume df is your cleaned DataFrame with all required columns
# Select a single record (row) by index, e.g., the first row
input_df = df.iloc[[0]]  # Note: double brackets to keep it as a DataFrame, not Series

# If your model requires specific preprocessing (e.g., encoding, scaling), apply it here
# For example, if you have a pipeline, just do:
# prediction = model.predict(input_df)

prediction = model.predict(input_df)

print("Prediction for selected input:", prediction)

probs = model.predict_proba(input_df)
print(probs)  #

Prediction for selected input: [1]
[[0.23 0.77]]


In [38]:
print(input_df)

     ID  Year_Birth   Education Marital_Status   Income  Kidhome  Teenhome  \
0  1826        1970  Graduation       Divorced  84835.0        0         0   

  Dt_Customer  Recency  MntWines  ...  Age  Total_Spending  Total_Purchases  \
0  2014-06-16        0       189  ...   55          1190.0               14   

   Education_enc  Marital_Status_enc  Country_enc  Age_Group  Has_Children  \
0            0.0                 0.0          0.0    Younger             0   

   Online_Purchases  Store_Purchases  
0                 8                6  

[1 rows x 39 columns]
